In [1]:
if __name__=='__main__':
    import notebookimport

In [2]:
import os
import sqlite3
import pandas as pd

In [3]:
if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='mexico18.db'
    conn = sqlite3.connect(dbname)
    rally='Mexico'

In [4]:
#!/Users/ajh59/anaconda3/bin/pip install tracery
import tracery
from tracery.modifiers import base_english
from inflectenglish import inflect_english

def pandas_row_mapper(row, rules, root,  modifiers=base_english):
    ''' Function to parse single row of dataframe '''
    row=row.to_dict()
    rules=rules.copy()

    for k in row:
        rules[k] = str(row[k])
        grammar = tracery.Grammar(rules)
        if modifiers is not None:
            if isinstance(modifiers,list):
                for modifier in modifiers:
                    grammar.add_modifiers(modifier)
            else:
                grammar.add_modifiers(modifiers)

    return grammar.flatten(root)

In [5]:
q='SELECT * FROM startlists st JOIN startlist_classes sc ON sc.entryid = st.entryid'
dd=pd.read_sql(q,conn)
dd.head()

,codriver.abbvName,codriver.code,codriver.country.countryId,codriver.country.iso2,codriver.country.iso3,codriver.country.name,codriver.countryId,codriver.firstName,codriver.fullName,codriver.lastName,...,tag,tag.name,tag.tagId,tagId,tyreManufacturer,vehicleModel,eventClassId,eventId,name,entryId
0,J. INGRASSIA,ING,76,FR,FRA,France,76,Julien,Julien INGRASSIA,INGRASSIA,...,None,None,NaN,NaN,Michelin,Fiesta WRC,137,28,RC1,1757
1,D. BARRITT,BAR,235,GB,GBR,United Kingdom of Great Britain and Northern I...,235,Daniel,Daniel BARRITT,BARRITT,...,None,None,NaN,NaN,Michelin,Fiesta WRC,137,28,RC1,1758
2,M. MARKKULA,MAR,75,FI,FIN,Finland,75,Mikko,Mikko MARKKULA,MARKKULA,...,None,None,NaN,NaN,Michelin,Fiesta WRC,137,28,RC1,1759
3,A. JAEGER-SYNNEVAAG,JAE,166,NO,NOR,Norway,166,Anders,Anders JAEGER-SYNNEVAAG,JAEGER-SYNNEVAAG,...,None,None,NaN,NaN,Michelin,i20 Coupe WRC,137,28,RC1,1760
4,N. GILSOUL,GIL,22,BE,BEL,Belgium,22,Nicolas,Nicolas GILSOUL,GILSOUL,...,None,None,NaN,NaN,Michelin,i20 Coupe WRC,137,28,RC1,1761


In [12]:
dd.columns

Index(['codriver.abbvName', 'codriver.code', 'codriver.country.countryId',
       'codriver.country.iso2', 'codriver.country.iso3',
       'codriver.country.name', 'codriver.countryId', 'codriver.firstName',
       'codriver.fullName', 'codriver.lastName', 'codriver.personId',
       'codriverId', 'driver.abbvName', 'driver.code',
       'driver.country.countryId', 'driver.country.iso2',
       'driver.country.iso3', 'driver.country.name', 'driver.countryId',
       'driver.firstName', 'driver.fullName', 'driver.lastName',
       'driver.personId', 'driverId', 'eligibility', 'entrant.entrantId',
       'entrant.logoFilename', 'entrant.name', 'entrantId', 'entryId',
       'eventId', 'group.name', 'groupId', 'group.groupId', 'identifier',
       'manufacturer.logoFilename', 'manufacturer.manufacturerId',
       'manufacturer.name', 'manufacturerId', 'priority', 'status', 'tag',
       'tag.name', 'tag.tagId', 'tagId', 'tyreManufacturer', 'vehicleModel',
       'eventClassId', 'eventId',

In [13]:
rules = {'origin': "#driver# and #codriver# #ffo# #entrant#.",
         'ffo':['for the', 'from the', 'of the'],
         'driver': "#driverfullName# (#drivercountryiso3#)",
         'codriver':[ "#codriverfullName# (#codrivercountryiso3#)", "co-driver #codriverfullName# (#codrivercountryiso3#)"],
         'car':"#manufacturername# #vehicleModel#",
        'entrant':"#entrantname#"}

In [18]:
dd.columns=[c.replace('.','') for c in dd.columns]
dd['report'] = dd.apply(lambda row: pandas_row_mapper(row, rules, "#origin#"), axis=1)


In [30]:
classes = pd.api.types.CategoricalDtype(categories = ['RC1', 'RGT', 'RC2', 'RC3', 'RC4','NRC1', 'NRC2','NRC3','NRC4'], 
                                            ordered = True)

In [32]:
dd['name'] = dd['name'].astype(classes)

grouped = dd.groupby('name')

for name, group in grouped:
    if len(group):
        print('Group {}:'.format(name))
        for i,r in group.head().iterrows():
            print(r['report'])

Group RC1:
Sébastien OGIER (FRA) and Julien INGRASSIA (FRA) of the M-SPORT FORD WORLD RALLY TEAM.
Elfyn EVANS (GBR) and Daniel BARRITT (GBR) for the M-SPORT FORD WORLD RALLY TEAM.
Teemu SUNINEN (FIN) and Mikko MARKKULA (FIN) from the M-SPORT FORD WORLD RALLY TEAM.
Andreas MIKKELSEN (NOR) and co-driver Anders JAEGER-SYNNEVAAG (NOR) of the HYUNDAI SHELL MOBIS WRT.
Thierry NEUVILLE (BEL) and Nicolas GILSOUL (BEL) for the HYUNDAI SHELL MOBIS WRT.
Group RC2:
Pontus TIDEMAND (SWE) and co-driver Jonas ANDERSSON (SWE) for the Skoda Motorsport.
Jari HUTTUNEN (FIN) and Antti LINNAKETO (FIN) for the Hyundai Motorsport.
Kalle ROVANPERÄ (FIN) and co-driver Jonne  HALTTUNEN (FIN) from the Skoda Motorsport.
Marco BULACIA WILKINSON (BOL) and co-driver Fernando MUSANO (ARG) for the Marco Bulacia Wilkinson.
Nil SOLANS BALDO (ESP) and Miquel  IBAÑEZ SOTOS (ESP) for the Nil Solans Baldo.
Group RC4:
Tom WILLIAMS (GBR) and co-driver Phil HALL (GBR) of the TOM WILLIAMS.
Group NRC2:
Miguel-Antonio GRANADOS CE